In [3]:
from collector_factory import CollectorFactory
from writer_factory import WriterFactory
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

c:\Users\Susana\anaconda3\envs\aa1\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 1. Collect and write data

In [2]:
collector = CollectorFactory.initialize_collector('APIncbi')
writer = WriterFactory.initialize_writer('FileWriter')
writer.set_path('./data')
list = []
n = 10000
for seq_object in collector.collect(n):
    writer.write(seq_object[0], str(seq_object[1]))
    list.append(seq_object[1])

NameError: name 'CollectorFactory' is not defined

In [3]:
unique_values = set(list)
print(len(unique_values))

3562


### 2.1 Encoding the data with One-Hot

In [4]:
aminoacids = 'ACDEFGHIKLMNPQRSTVWYXBJU'
amino_dict = {amino: idx for idx, amino in enumerate(aminoacids)}

def one_hot_encode(secuencia, max_length):
    encoding = np.zeros((max_length, len(aminoacids)))
    
    for i, amino in enumerate(secuencia):
        if amino not in amino_dict:
            print("Unrecognized amino acid:", amino)
        else:
            encoding[i, amino_dict[amino]] = 1
    return encoding.flatten()

data = []

for file in os.listdir('./data'):
    with open(os.path.join('./data', file), 'r') as f:
        seq = f.read().strip()
        data.append({'filename': file, 'sequence': seq})

max_length = max(len(record['sequence']) for record in data)

for record in data:
    encoded_seq = one_hot_encode(record['sequence'], max_length)
    record['encoded_sequence'] = encoded_seq

df = pd.DataFrame(data)

display(df.head())

,filename,sequence,encoded_sequence
0,aab01987.1.bio,MRLLAKIICLMLWAICVAE,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,aab93671.1.bio,LNRVVGGEDSTDSEWPWIVSIQKNGTHHCAGSLLTSRWVITAAHCF...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,aac04477.1.bio,PPALAMPMPTEGTPPPLSGTPIPVPAYFRHAEPGFSLKRPRGLSRS...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,aac08447.1.bio,MGITGNTSPFGQPFSQAGGQPMGATGVNPQLASKQSMVNSLPTFPT...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,aac08452.1.bio,DDQLLDDGKTLGECGFTSQTARPQAPATVGLAFRADDTFEALCIEP...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### 2.2. Encoding the data  by Physicochemical Properties

In [9]:
# IMPRIMIR EL ENCODED_SEQ de la fila 0
len(df['encoded_sequence'][0])


122904

### 3. Unsupervised Learning

### 3.1. DBSCAN

In [4]:
X = np.array(df['encoded_sequence'].tolist())

X_scaled = StandardScaler().fit_transform(X)

dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(X_scaled)

df['cluster'] = clusters

num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
print("Clusters:", num_clusters)


Clusters: 331


### 3.2. K-means

In [5]:
X = np.array(df['encoded_sequence'].tolist())

inertia = []
k_values = range(12, 25)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)


KeyboardInterrupt: 

In [1]:
plt.figure(figsize=(10, 6))
plt.plot(k_values, inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_values)
plt.grid()
plt.show()

NameError: name 'plt' is not defined

### 4. Analysis of results